In [1]:
import numpy as np
import WLCgreen as wlc
import propagator 
import special as sp
from numba import jit
import MultiPoint as mp
import imp  # imp.reload(module)

import matplotlib.pyplot as plt
%matplotlib
#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()

Using matplotlib backend: Qt5Agg


In [2]:
def plotlog(x, y, alpha, xrange=1, numx=2):
    xv = np.logspace(np.log10(x), np.log10(x) + xrange, numx)
    C = y / (x**alpha)
    yv = C * xv**(alpha)
    plt.loglog(xv, yv,'k--')

## 2 point

In [3]:
nlam=3
ORDEig=25
d=3
mu=0

nkpts=500
K=np.logspace(-5,7,nkpts)
props=[] # list of propagators at each K
for ii in range(0,nkpts):
    name = ii
    props.append(propagator.propagator(name,K[ii],mu,nlam=nlam))

### AA

In [4]:
fa=0.5
N=[0.1,1.0,10,100,1000,10000,100000]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    lam=0
    lam0=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IAAresum(N[nn],fa,lam0,lam,props[ii])
        dub[ii,nn]=mp.IAAexplicit(N[nn],fa,lam0,lam,props[ii])

In [6]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real*np.sqrt(N[ii]),abs(ana[:,ii])/(N[ii]**2),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,abs(ana[:,ii]),\
    #            color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real*np.sqrt(N[ii]),abs(dub[:,ii])/(N[ii]**2),'--',\
                color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
    #ax.loglog(K.real,abs(dub[:,ii]),'--',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-3,-1) 
ax.set_xlabel('K')
ax.set_title('AA')
ax.set_ylim([10**-8,10**10])
plt.show()

In [ ]:
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.5
for ii in range(0,nkpts):
    lam=0
    lam0=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IABresum(N[nn],fa,lam0,lam,props[ii])
        dub[ii,nn]=mp.IABexplicit(N[nn],fa,lam0,lam,props[ii])

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii]),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real,abs(dub[:,ii]),'--',\
                color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-7,-2)
ax.set_xlabel('K')
ax.set_title('AB')
ax.set_ylim([10**-15,100000])
plt.show()

## 3 Point

In [3]:
nlam=5
ORDEig=3
d=3
mu=0

nkpts=2000
K=np.logspace(-4,7,nkpts)
props1=[] # list of propagators at each K
props2=[] # list of propagators at each K
for ii in range(0,nkpts):

    name=ii
    props1.append( propagator.propagator(name,K[ii],mu,nlam=nlam) )

    name=ii+0.5
    props2.append( propagator.propagator(name,K[ii],mu,nlam=nlam) )

#### AAA

In [6]:
fa=0.5
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=0
    lam_2=0
    lam0_2=0
    for nn in range(0,len(N)):
        #IAAAexplicit(N,fa,lam0_1,lam_1,lam0_2,lam_2,p1,p2)
        ana[ii,nn]=mp.IAAA(N[nn],fa,\
                                lam0_1,lam_1,\
                                lam0_2,lam_2,\
                                props1[ii],props2[ii])
        dub[ii,nn]=mp.IAAAexplicit(N[nn],fa,\
                                    lam0_1,lam_1,\
                                    lam0_2,lam_2,\
                                    props1[ii],props2[ii])

In [7]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii].real),\
                color=[col, 0.0, 1-col],label='N='+str(N[ii]))
    #ax.loglog(K.real,abs(dub[:,ii]),':',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend()#bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-13,-3)
ax.set_xlabel('K')
#ax.set_title('A'+str(lam_1)+',A'+str(lam_2)+',A'+str(lam0_2))
ax.set_title('\int d^2s G_11(K) G_11(K)')
ax.set_ylim([10**-18,10**2])
print(lam_1,',',lam_2,',',lam0_2)
plt.show()

0 , 0 , 0


In [ ]:
nl=5
out=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    for l in range(0,(nl)):
        out[ii,l]=props1[ii].res[l][lam0_1,lam_1]

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,nl):
    col=float(ii)/nl
    ax.loglog(K.real,abs(out[:,ii].real),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
ax.set_xlabel('K')
ax.set_title('res')
print('lam',lam_1,'lam0',lam_0)
ax.set_ylim([10**-25,10])
plt.show()

In [ ]:
nl=5
out=np.zeros((nkpts,len(N),nl),dtype=type(1+1j))
for ii in range(0,nkpts):
    for nn in range(0,len(N)):
        for l in range(0,(nl)):
            out[ii,nn,l]=props1[ii].eig[l]

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,nl):
    col=float(ii)/nl
    ax.loglog(K.real,abs(out[:,0,ii].real),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
ax.set_xlabel('K')
ax.set_title('eig')

plt.show()

#### ABB

In [ ]:
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.4466
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=0
    lam_2=0
    lam0_2=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IABBresum(N[nn],fa,\
                                lam0_1,lam_1,\
                                lam0_2,lam_2,\
                                props1[ii],props2[ii])
        #dub[ii,nn]=IAAAexplicit(N[nn],lam0,lam,props1[ii],props2[ii])

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii]),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,abs(dub[:,ii]),'--',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-9,-3)
ax.set_xlabel('K')
ax.set_title('ABB')
ax.set_ylim([10**-18,10**15])
plt.show()

## 4 point

In [3]:
nlam=5
ORDEig=25
d=3
mu1=0
mu2=0
mu3=0

nkpts=100
K=np.logspace(-2,5,nkpts)
props1=[] # list of propagators at each K
props2=[] # list of propagators at each K
props3=[] # list of propagators at each K
for ii in range(0,nkpts):

    name=ii
    props1.append( propagator.propagator(name,1.0*K[ii],mu1,nlam=nlam) )

    name=ii+0.5
    props2.append( propagator.propagator(name,2.0*K[ii],mu2,nlam=nlam) )
    
    name=ii+0.25
    props3.append( propagator.propagator(name,1.0*K[ii],mu3,nlam=nlam) )

In [20]:
N=[0.001,0.01,0.1,1.0,10,100,1000]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.4466
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=0
    lam_2=0
    lam0_2=0    
    lam_3=0
    lam0_3=0
    
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IAAAA(N[nn],fa,\
                                 lam0_1,lam_1,\
                                 lam0_2,lam_2,\
                                 lam0_3,lam_3,\
                                 props1[ii],props2[ii],props3[ii],include_b=1)
        if np.isnan(ana[ii,nn]):
            print('Nan Found')
        #dub[ii,nn]=mp.IAAAAexplicit(N[nn],fa,\
        #                         lam0_1,lam_1,\
        #                         lam0_2,lam_2,\
        #                         lam0_3,lam_3,\
        #                         props1[ii],props2[ii],props3[ii])

In [21]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,ana[:,ii].real,'-',\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real,-ana[:,ii].real,'--',\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,-dub[:,ii].real,':',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
#ax.legend(bbox_to_anchor=(1.05,1),loc=2)
#ax.legend(bbox_to_anchor=(1.05,1))
plotlog(2e3,1e-4,-1)
plotlog(2e3,1e-5,-2)
plotlog(2e3,1e-6,-3)
plotlog(2e3,1e-7,-4)
ax.set_xlabel('K')
ax.set_title('AAAA')
#ax.set_ylim([10**-18,10**15])
plt.show()